<h1> NEAT Implementation Practice </h1>
My Raw NEAT implementation, specifically for the cartpole environment.

# Some Notes
Here are some things about which knobs you can turn in each cell:
1. Imports. Don't touch anything there unless you wanna try something out at your own risk.
2. Environment Declarations. Change the envName string if you want to test out other environments, but just note that you may need to change some other parameters in the evaluation cell as well if you do so.
3. Hyperparameters. Change as you please, but I would recommend sticking to the values from the NEAT paper and/or neatevolve.lua.
4. Some functions. I wouldn't touch this.
5. Genome Class. I wouldn't touch this, but please feel free to look at the mess I've made. I have recently moved the crossover and delta functions as static methods to this class.
8. Evaluation. As mentioned in 2), if you change the environment, then you made need to change the evaluateAction method of the genome or some of the target fitness requirements for episode termination.

In [1]:
import gym
import numpy as np # might not even need this ?
import random # pseudo-random|
import math
import time # for timing how long it runs for 
import PySimpleGUI as sg
import pickle 
import matplotlib.pyplot as plt
import copy

In [2]:
%matplotlib qt

In [3]:
# first define the environment and get the both the action and observation spaces 
envName = 'CartPole-v1' # just need to change this if you wanna try it in environments
env = gym.make(envName)
plt.title(envName + ' Average Fitness across Generations - RNN')
# simple gui stuff
sg.theme = ('DarkAmber')
layout = [  [sg.Text('Trial: '), sg.Text(0, key='TRIAL-NUM', size= (6, 1))],
            [sg.Text('Simulation Info'), sg.Text(''),sg.Text('BestGenome Info')],
            [sg.Text('Generation: '), sg.Text(0, key='GENER-NUM', size = (6,1)), sg.Text('Generation: '), sg.Text(0, key='BEST-GENER-NUM', size = (6,1))],
            [sg.Text('Genome: '), sg.Text(0, key='GENOM-NUM',size = (6,1)), sg.Text('Generation: '), sg.Text(0, key='BEST-GENOM-NUM', size = (6,1))],
            [sg.Text('Species: '), sg.Text(0, key='SPECIES-NUM',size = (6,1)), sg.Text('Species: '), sg.Text(0, key='BEST-SPECIES-NUM',size = (6,1))],
            [sg.Text('Fitness: '), sg.Text(0, key='FITNESS-NUM',size = (6,1)), sg.Text('Fitness: '), sg.Text(0, key='BEST-FITNESS-NUM',size = (6,1))]]



# # print out the observation and action spaces
# print(env.action_space.shape[0]) # number of valid actions we can take 
# print(env.observation_space.shape[0]) # number of "sensors" of the environment that we have (number of inputs)

# NOTE: Need to keep track of the valid values these can take 
# NOTE: Also need to have inputs + 1, with the 1 being the bias (stc?)
# NOTE: changes between Box() and Discrete() depending on environment
outputs = env.action_space.n
inputs = env.observation_space.shape[0] + 1


In [4]:
# hyperparameters, according to section 4.1 Parameter Settings (changed some parameters for myself)
deltaThreshold = 1.0

c1 = c2 = 2.0
c3 = 0.4
perturbVal = 0.025


numUntilStagnant = 15


genomeMutationChance = 0.25 # neatevolve.lua has this pretty low 
connectionUniformPerturb = 0.9
connectionRandomValue = 1 - connectionUniformPerturb
disableGeneChance = 0.75 
mutateNoCrossoverChance = 0.25
interspeciesMatingRate = 0.01 
smallPopulationNewNodeChance = 0.05
largePopulationNewNodeChance = 0.50 # using these right now 
smallPopulationNewConnectionChance = 0.03
largePopulationNewConnectionChance = 0.3 # using these right now 

In [5]:
# activation functions 
def sigmoid(x):
    return 1/(1+np.exp(-x))

def steepsigmoid(x):
    return 1/(1+np.exp(-4.9*x))

In [6]:
class Genome:
    globalInnovationNumber = 1
    historyMaxLength = 10
    def __init__(self, inputs=1, outputs=1):
        self.nodeGenes={}
        self.connectGenes={}
        self.inputNodes=[]
        self.outputNodes=[]
        self.hiddenNodes=[]
        self.fitness=0.0
        self.sharedFitness=0.0
        self.species=0
        self.inputCount=inputs
        self.outputCount=outputs
        self.mutationsuccess=False
    
    
    # should be an EXACT copy (only necessary when copying the champions)
    @staticmethod
    def copyGenome(genome):
        g = Genome(genome.inputCount, genome.outputCount)
        g.nodeGenes = copy.deepcopy(genome.nodeGenes)
        g.connectGenes =  copy.deepcopy(genome.connectGenes)
        
        g.inputNodes = copy.deepcopy(genome.inputNodes)
        g.hiddenNodes = copy.deepcopy(genome.hiddenNodes)
        g.outputNodes = copy.deepcopy(genome.outputNodes)
        
        g.clearNodeValues()
        g.clearHistoryLists()
        
        return g
    
    @staticmethod
    def resetGlobalInnovationNumber():
        Genome.globalInnovationNumber = 1
        
    @staticmethod
    def crossover(g1, g2):
        # print('called crossover')
        if g2.fitness > g1.fitness:
            temp = g1
            g1 = g2
            g2 = temp
            
        g = Genome(g1.inputCount, g1.outputCount)
        g1GenomeList = {}
        for c in g1.connectGenes:
            g1GenomeList[g1.connectGenes[c]['historicalMarker']]={'connection':c, 'weight': g1.connectGenes[c]['weight'],'isExpressed': g1.connectGenes[c]['isExpressed']}
        g2GenomeList = {}
        for c in g2.connectGenes:
            g2GenomeList[g2.connectGenes[c]['historicalMarker']]={'connection':c, 'weight': g2.connectGenes[c]['weight'],'isExpressed': g2.connectGenes[c]['isExpressed']}
            
        for i in range(max(len(g1GenomeList), len(g2GenomeList))):
            if (i+1 in g1GenomeList and i+1 in g2GenomeList):
                if (random.random() <= 0.50): # inherit from genome1
                    g.connectGenes[g1GenomeList[i+1]['connection']] = {'historicalMarker':i+1, 'weight':g1GenomeList[i+1]['weight'], 'isExpressed':g1GenomeList[i+1]['isExpressed']}
                else: # inherit from genome2 
                    g.connectGenes[g2GenomeList[i+1]['connection']] = {'historicalMarker':i+1, 'weight':g2GenomeList[i+1]['weight'], 'isExpressed':g2GenomeList[i+1]['isExpressed']}
                if (g1GenomeList[i+1]['isExpressed'] == False or g2GenomeList[i+1]['isExpressed'] == False):
                    if (random.random() <= disableGeneChance):
                        g.connectGenes[g1GenomeList[i+1]['connection']]['isExpressed'] = False
                    else:
                        # print('enabled')
                        g.connectGenes[g1GenomeList[i+1]['connection']]['isExpressed'] = True
            if (i+1 in g1GenomeList and i+1 not in g2GenomeList):
                g.connectGenes[g1GenomeList[i+1]['connection']] = {'historicalMarker':i+1, 'weight':g1GenomeList[i+1]['weight'], 'isExpressed':g1GenomeList[i+1]['isExpressed']}
        
        # at this point, you have all of the connections. now put in all of the nodes 
        g.initGenome() # first insert the input and output nodes
        
        for c in g.connectGenes:
            if (c[0] not in g.inputNodes and c[0] not in g.outputNodes):
                    g.hiddenNodes.append(c[0])
                    if c[0] in g1.nodeGenes:
                        g.nodeGenes[c[0]] = g1.nodeGenes[c[0]]
                    elif c[0] in g2.nodeGenes:
                        g.nodeGenes[c[0]] = g2.nodeGenes[c[0]]
            if (c[1] not in g.inputNodes and c[1] not in g.outputNodes):
                    g.hiddenNodes.append(c[1])
                    if c[1] in g1.nodeGenes:
                        g.nodeGenes[c[1]] = g1.nodeGenes[c[1]]
                    elif c[1] in g2.nodeGenes:
                        g.nodeGenes[c[1]] = g2.nodeGenes[c[1]]
                        
        g.hiddenNodes = list(set(g.hiddenNodes))                 
        g.clearAdjacentNodesList()
        
        for c in g.connectGenes:
            if (g.connectGenes[c]['isExpressed'] == True):
                g.nodeGenes[c[1]]['adjacentNodes'].append(c[0])
    
        g.cleanANL()
        
        return g
            
    
    @staticmethod
    def delta(genome1, genome2, c1=1.0, c2=1.0, c3=0.4):
        if (not genome1.connectGenes or not genome2.connectGenes):
            return 0
        connectionMismatch = []
        for connection1 in genome1.connectGenes:
            if (connection1 not in genome2.connectGenes):
                connectionMismatch.append(connection1)
        for connection2 in genome2.connectGenes:
            if (connection2 not in genome1.connectGenes):
                connectionMismatch.append(connection2)
        diffsum = 0
        num = 0
        for connection1 in genome1.connectGenes:
            if (connection1 not in connectionMismatch):
                diffsum += abs(genome1.connectGenes[connection1]['weight'] -
                               genome2.connectGenes[connection1]['weight'])
                num += 1
        averageWeightDifference = diffsum / num
        #print(averageWeightDifference)
        N = 0
        if len(genome1.connectGenes) >= len(genome2.connectGenes):
            N = len(genome1.connectGenes)
        else:
            N = len(genome2.connectGenes)
        # print(((len(connectionMismatch)*(c1 + c2))/N) + (c3*averageWeightDifference))
        return ((len(connectionMismatch) *
                 (c1 + c2)) / N) + (c3 * averageWeightDifference)
    
    def clearHistoryLists(self):
        for node in self.nodeGenes:
            self.nodeGenes[node]['valueHistory'].clear()
    
    def cleanANL(self):
        for node in self.nodeGenes:
            self.nodeGenes[node]['adjacentNodes'] = list(set(self.nodeGenes[node]['adjacentNodes']))
    
    def clearAdjacentNodesList(self):
        for node in self.nodeGenes:
            self.nodeGenes[node]['adjacentNodes'].clear()
    
    def clearNodeValues(self):
        for node in self.nodeGenes:
            self.nodeGenes[node]['value'] = -2.0 # sentinel value 
    
    def initGenome(self):
        for i in range(self.inputCount):
            self.insertNode('input')
        for i in range(self.outputCount):
            self.insertNode('output')
            
    def initRandomGenome(self):
        Genome.resetGlobalInnovationNumber()
        self.initGenome()
        for i in range(self.inputCount):
            for j in range(self.outputCount):
                self.insertConnection(outNode=i+1, inNode=self.inputCount+j+1, weight=random.uniform(-1,1))
        # print(self.nodeGenes)
    
    def evaluateSharedFitness(self, speciesCount):
        self.sharedFitness = self.fitness/ speciesCount
    
    def resetFitness(self):
        self.fitness = 0.0
        self.sharedFitness = 0.0
        
    def insertNode(self, nodeType='input', value=0.0):
        self.nodeGenes[len(self.nodeGenes)+1]={'type': nodeType, 'value': value, 'valueHistory':[], 'adjacentNodes':[]}
        if (nodeType == 'input'):
            self.inputNodes.append(len(self.nodeGenes))
        elif (nodeType == 'output'):
            self.outputNodes.append(len(self.nodeGenes))
        else:
            self.hiddenNodes.append(len(self.nodeGenes))
    
    def insertConnection(self, outNode, inNode, weight = 0.5, isExpressed = True):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) not in self.connectGenes and inNode != 1: # check if the connection already exists, and do not feed into the bias cell
                self.connectGenes[(outNode, inNode)] = {'historicalMarker': Genome.globalInnovationNumber, 'weight': weight, 'isExpressed': isExpressed}
                self.nodeGenes[inNode]['adjacentNodes'].append(outNode)
                Genome.globalInnovationNumber += 1
                self.mutationSuccess = True
                
    def disableConnection(self, outNode, inNode):
        if outNode in self.nodeGenes and inNode in self.nodeGenes:
            if (outNode, inNode) in self.connectGenes and self.connectGenes[(outNode, inNode)]['isExpressed'] == True:
                # print('disabling ({},{})'.format(outNode, inNode))
                self.connectGenes[(outNode, inNode)]['isExpressed']=False
                self.nodeGenes[inNode]['adjacentNodes'].remove(outNode)
                
    def mutateAddConnection(self, outNode, inNode):
        self.insertConnection(outNode, inNode, random.uniform(-1, 1), True)
        
    def mutateAddRandomConnection(self):
        timesAttemptedToConnected = 0
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            self.mutateAddConnection(random.choice(list(self.nodeGenes)),random.choice(list(self.nodeGenes)))
            timesAttemptedToConnected +=1
            if (timesAttemptedToConnected > 100):
                print('Attempted to connect too many times. probably already full')
                return
        self.mutationSuccess = False
    
    def mutateAddNode(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes and self.connectGenes[(outNode, inNode)]['isExpressed'] == True:
                self.insertNode('hidden', 0.0)
                # disable old connection'
                self.disableConnection(outNode, inNode)
                self.insertConnection(outNode, len(self.nodeGenes), 1.0, True)
                # get original weight 
                originalWeight = self.connectGenes[(outNode, inNode)]['weight']
                self.insertConnection(len(self.nodeGenes), inNode, originalWeight, True)
                self.mutationSuccess = True

    def mutateAddRandomNode(self):
        if (not self.connectGenes): # need to debug this 
            return
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            randomTuple = random.choice(list(self.connectGenes))
            self.mutateAddNode(randomTuple[0],randomTuple[1])
        self.mutationSuccess = False
    
    
    # helper to nodeRecur
    def recurrentConnection(self, originalNode, currentNode):
        if (originalNode == currentNode):
            return True
        if (currentNode in self.inputNodes):
            return False 
        for node in self.nodeGenes[currentNode]['adjacentNodes']:
            if ((node, currentNode) in self.connectGenes):
                if (node != currentNode): # not interested in self-loops
                    if(self.recurrentConnection(originalNode, node)):
                        return True
        return False
    
    # helper to evaluateAction
    def nodeRecur(self, nodeKey):
        nodeSum = 0
        for n in self.nodeGenes[nodeKey]['adjacentNodes']:
            if (self.connectGenes[(n, nodeKey)]['isExpressed'] == True):
                if (self.nodeGenes[n]['type'] == 'input' or self.nodeGenes[n]['value'] != -2): # input or already evaluated
                    nodeSum += self.nodeGenes[n]['value'] * self.connectGenes[(n, nodeKey)]['weight']
                elif (self.recurrentConnection(nodeKey, n)): # recurrent case
                    nodeSum += self.connectGenes[(n, nodeKey)]['weight'] * sum(self.nodeGenes[n]['valueHistory'])
                elif (self.nodeGenes[n]['value'] == -2): # unevaluted (recursive case)
                    nodeSum += self.nodeRecur(n) * self.connectGenes[(n, nodeKey)]['weight']
        
        if (self.nodeGenes[nodeKey]['type'] == 'output'):
            # print('output value: {}'.format(np.exp(nodeSum)))
            return np.exp(nodeSum)
        else:
            return steepsigmoid(nodeSum)
    
    def evaluateAction(self, observation):
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1]['value'] = 1.0
            
            for i in range(self.inputCount-1):
                self.nodeGenes[i+2]['value'] = observation[i]
                
                # TODO: Look into this error:
                for n in self.nodeGenes[i+2]['adjacentNodes']:
                    if (self.connectGenes[(n, i+2)]['isExpressed'] == True):
                        self.nodeGenes[i+2]['value'] += self.connectGenes[(n,i+2)]['weight']*sum(self.nodeGenes[n]['valueHistory'])
                
            for h in self.hiddenNodes:
                if(self.nodeGenes[h]['value'] == -2.0):
                    self.nodeGenes[h]['value'] = self.nodeRecur(h)
                    
            softmaxsum = 0.0
            for o in self.outputNodes:
                self.nodeGenes[o]['value'] = self.nodeRecur(o)
                # print(self.nodeGenes[o]['value'])
                softmaxsum += self.nodeGenes[o]['value']

            output = []
            # before we evaluate, go back and update the history of each node
            for node in self.nodeGenes:
                if self.nodeGenes[node]['type'] == 'output':
                    historyValue = self.nodeGenes[node]['value'] / softmaxsum
                    output.append(historyValue)
                else:
                    historyValue = self.nodeGenes[node]['value']
                # check if the history is full
                if (len(self.nodeGenes[node]['valueHistory']) == Genome.historyMaxLength): # if it is, pop the back and insert into the front
                    self.nodeGenes[node]['valueHistory'].pop()
                    self.nodeGenes[node]['valueHistory'].insert(0, historyValue)
                else:
                    self.nodeGenes[node]['valueHistory'].insert(0, historyValue)
            return output.index(max(output))        
        
    def evaluateActionBox(self, observation):
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1]['value'] = 1.0
            
            for i in range(self.inputCount-1):
                self.nodeGenes[i+2]['value'] = observation[i]
                
                # TODO: Look into this error:
                for n in self.nodeGenes[i+2]['adjacentNodes']:
                    if (self.connectGenes[(n, i+2)]['isExpressed'] == True):
                        self.nodeGenes[i+2]['value'] += self.connectGenes[(n,i+2)]['weight']*sum(self.nodeGenes[n]['valueHistory'])
                
            for h in self.hiddenNodes:
                if(self.nodeGenes[h]['value'] == -2.0):
                    self.nodeGenes[h]['value'] = self.nodeRecur(h)
                    
            softmaxsum = 0.0
            for o in self.outputNodes:
                self.nodeGenes[o]['value'] = self.nodeRecur(o)
                # print(self.nodeGenes[o]['value'])
                softmaxsum += self.nodeGenes[o]['value']

            output = []
            # before we evaluate, go back and update the history of each node
            for node in self.nodeGenes:
                if self.nodeGenes[node]['type'] == 'output':
                    historyValue = self.nodeGenes[node]['value'] / softmaxsum
                    output.append(historyValue)
                else:
                    historyValue = self.nodeGenes[node]['value']
                # check if the history is full
                if (len(self.nodeGenes[node]['valueHistory']) == Genome.historyMaxLength): # if it is, pop the back and insert into the front
                    self.nodeGenes[node]['valueHistory'].pop()
                    self.nodeGenes[node]['valueHistory'].insert(0, historyValue)
                else:
                    self.nodeGenes[node]['valueHistory'].insert(0, historyValue)
            return output

> connection weights mutate as in any NE system, with each connection either perturbed or not at each generation 

In [ ]:
maxGenerations = 50
populationLimit = 150
championThresholdCount = 5

inputs = env.observation_space.shape[0] + 1
outputs = env.action_space.n

bestGenomeDictionary = {}

targetFitness = 500
maxTestTrials = 100
targetAverageFitness = 495.5

window = sg.Window(envName + ' - RNN', layout, finalize=True)
start_time = time.time()

globalEvaluationCounter = 0
solvedCounter = 0

for q in range(maxTestTrials):
    hl, = plt.plot([],[])
    window['TRIAL-NUM'].update(q+1)
    species = {}
    pop = {}
    solved = False
    trialLoopCounter = 0
    bestGenome = None
    highestFitness = float("-inf")
    print('Beginning Trial #{}'.format(q+1))
    for gen in range(maxGenerations):
        genFitness = 0.0
        print('Beginning Generation #{}'.format(gen+1))
        window['GENER-NUM'].update(gen+1)
        if (gen == 0 and not pop):
            for genome in range(populationLimit):
                # print('loop')
                g = Genome(inputs, outputs)
                g.initRandomGenome()
                pop[genome+1]=g
        
        for genome in pop:
            print(genome)
            window['GENOM-NUM'].update(genome)
            if (not species):
                species[len(species)+1] = {'summedSharedFitness': 0.0, 'maxFitness': 0.0, 'highestSharedFitness': 0.0, 'stagnantCounter': 0, 'isStagnant': False, 'list' : [pop[genome]]}
                pop[genome].species = len(species)
            else:
                for og in species:
                    if (Genome.delta(pop[genome], species[og]['list'][0], c1, c2, c3) < deltaThreshold): # just compare with the first species
                        species[og]['list'].append(pop[genome])
                        pop[genome].species = og
                        break
                    else:
                        if (og == len(species)):
                            species[len(species)+1] ={'summedSharedFitness': 0.0, 'maxFitness': 0.0, 'highestSharedFitness': 0.0, 'stagnantCounter': 0, 'isStagnant': False, 'list' : [pop[genome]]}
                            pop[genome].species = len(species)
                            break
            
            window['SPECIES-NUM'].update(pop[genome].species)
            pop[genome].resetFitness()
            pop[genome].clearHistoryLists()
            observation = env.reset()
            t = 0
            trialLoopCounter += 1 # **
            while(True):
                t+=1
                env.render()
                action = pop[genome].evaluateAction(observation)
                observation, reward, done, info = env.step(action)
                pop[genome].fitness += reward
                window['FITNESS-NUM'].update(pop[genome].fitness)
                window.finalize()
                if done:
                    genFitness += pop[genome].fitness
                    if pop[genome].fitness > species[pop[genome].species]['maxFitness']:
                        species[pop[genome].species]['maxFitness'] = pop[genome].fitness
                        species[pop[genome].species]['stagnantCounter'] = 0
                        species[pop[genome].species]['isStagnant'] = False
                        species[pop[genome].species]['bestGenome'] = pop[genome]
                    if pop[genome].fitness > highestFitness:
                        bestGenome = pop[genome]
                        highestFitness = pop[genome].fitness
                        # update the window values 
                        window['BEST-GENER-NUM'].update(gen+1)
                        window['BEST-GENOM-NUM'].update(genome)
                        window['BEST-SPECIES-NUM'].update(pop[genome].species)
                        window['BEST-FITNESS-NUM'].update(pop[genome].fitness)
                        window.finalize()
                    if pop[genome].fitness >= targetFitness: # changes depending on environment !
                        print('Found an optimal genome. testing it...')
                        testBenchTotalFitness = 0
                        for r in range(maxTestTrials):
                            m = 0
                            pop[genome].resetFitness()
                            pop[genome].clearHistoryLists()
                            observation = env.reset()
                            while(True):
                                m+=1
                                env.render()
                                action = pop[genome].evaluateAction(observation)
                                observation, reward, done, info = env.step(action)
                                pop[genome].fitness += reward
                                window['FITNESS-NUM'].update(pop[genome].fitness)
                                window.finalize()
                                if done:
                                    testBenchTotalFitness+=pop[genome].fitness
                                    break
                        print('average fitness across 100 trials:{}'.format(testBenchTotalFitness/maxTestTrials))
                        if (testBenchTotalFitness/maxTestTrials >= targetAverageFitness):
                            print('found solution genome')
                            solved = True
                            solvedCounter +=1
                            bestGenomeDictionary[len(bestGenomeDictionary)+1] = pop[genome]
                    break
            if (solved):
                break
        if (solved):
            break
            
        hl.set_xdata(np.append(hl.get_xdata(), gen))
        hl.set_ydata(np.append(hl.get_ydata(), genFitness/populationLimit))
        plt.plot(hl.get_xdata(), hl.get_ydata())
        plt.draw()
        
        
# --------------------------------------------------------------------------------
        totalAdjustedFitness = 0.0
        for genome in pop:
            pop[genome].evaluateSharedFitness(len(species[pop[genome].species]))
            species[pop[genome].species]['summedSharedFitness'] += pop[genome].sharedFitness
            if pop[genome].sharedFitness > species[pop[genome].species]['highestSharedFitness']:
                species[pop[genome].species]['highestSharedFitness'] = pop[genome].sharedFitness
                species[pop[genome].species]['bestGenome'] = pop[genome]
                
        for s in species:
            totalAdjustedFitness += species[s]['summedSharedFitness']
            
        newPop = {}
        
        y = 0
        for s in species:
            if len(species[s]['list']) > championThresholdCount:
                newPop[y+1] = Genome.copyGenome(species[s]['bestGenome'])
                y += 1
        
        totalBreedCount = 0
        for s in species:
            if species[s]['isStagnant']:
                species[s]['breedAmountAllowed'] = 0
            else:
                species[s]['breedAmountAllowed'] = np.floor((species[s]['summedSharedFitness']/totalAdjustedFitness)*(populationLimit-y)).astype(int)
                totalBreedCount += species[s]['breedAmountAllowed']
        leftovers = populationLimit - y - totalBreedCount
        
        for s in species:
            amountToCull = np.floor(.75*len(species[s]['list'])).astype(int)
            species[s]['list'].sort(reverse = True, key=lambda genome: genome.fitness)
            for b in range(amountToCull):
                tempGenome = species[s]['list'].pop() 
        
        for s in species:
            for j in range(species[s]['breedAmountAllowed']):
                if (random.random() <= mutateNoCrossoverChance or len(species[s]['list'])==1):
                    g = Genome.copyGenome(random.choice(species[s]['list']))
                else:
                    g1 = random.choice(species[s]['list'])
                    g2 = random.choice(species[s]['list'])
                    g = Genome.crossover(g1, g2)
                
                # mutate 
                if (g.connectGenes):
                    if (random.random() <= genomeMutationChance):
                        for c in g.connectGenes:
                            if random.random() <= connectionUniformPerturb:
                                g.connectGenes[c]['weight'] += random.uniform(-perturbVal,perturbVal)
                            else:
                                g.connectGenes[c]['weight'] = random.uniform(-1.0, 1.0)
                                
                if (random.random() <= largePopulationNewNodeChance):
                    g.mutateAddRandomNode()
                if (random.random() <= largePopulationNewConnectionChance):
                    g.mutateAddRandomConnection()
                
                g.cleanANL()
                newPop[y+1] = g
                y+=1
                
        for k in range(leftovers):
            if (random.random() <= interspeciesMatingRate):
                speciesKey1 = random.choice(list(species))
                speciesKey2 = random.choice(list(species))
                g1 = random.choice(species[speciesKey1]['list'])
                g2 = random.choice(species[speciesKey2]['list'])
                g = Genome.crossover(g1, g2)
                if (g.connectGenes):
                    if (random.random() <= genomeMutationChance):
                        for c in g.connectGenes:
                            if (random.random() <= connectionUniformPerturb):
                                g.connectGenes[c]['weight'] += random.uniform(-perturbVal, perturbVal)
                            else:
                                g.connectGenes[c]['weight'] = random.uniform(-1.0, 1.0)
                if (random.random() <= largePopulationNewNodeChance):
                    g.mutateAddRandomNode()
                if (random.random() <= largePopulationNewConnectionChance):
                    g.mutateAddRandomConnection()
                
                g.cleanANL()
                newPop[y+1]= g
                y+=1
            else:
                speciesKey = random.choice(list(species))
                if (random.random() <= mutateNoCrossoverChance or len(species[speciesKey]['list'])==1):
                    g = Genome.copyGenome(random.choice(species[speciesKey]['list']))
                else:
                    g1 = random.choice(species[speciesKey]['list'])
                    g2 = random.choice(species[speciesKey]['list'])
                    g = Genome.crossover(g1, g2)
                if (g.connectGenes):
                    if (random.random() <= genomeMutationChance):
                        print('mutation genome connections !')
                        for c in g.connectGenes:
                            if (random.random() <= connectionUniformPerturb):
                                g.connectGenes[c]['weight'] += random.uniform(-perturbVal, perturbVal)
                            else:
                                g.connectGenes[c]['weight'] = random.uniform(-1.0, 1.0) 
                if (random.random() <= largePopulationNewNodeChance):
                    g.mutateAddRandomNode()
                if (random.random() <= largePopulationNewConnectionChance):
                    g.mutateAddRandomConnection()
                
                g.cleanANL()
                newPop[y+1] = g
                y+=1
                
# ---------------------------------------------------------------------------------------------
        if (gen >= 0):
            for s in species:
                species[s] = {'summedSharedFitness': 0.0, 'maxFitness': species[s]['maxFitness'], 'highestSharedFitness': 0.0, 'stagnantCounter': species[s]['stagnantCounter']+1, 'isStagnant': False, 'list' : [random.choice(species[s]['list'])]}
                if species[s]['stagnantCounter'] >= numUntilStagnant:  # if the maximum fitness of a species did not improve in 15 generations, the networks in the stagnant species were not allowed to reproduce 
                    species[s]['isStagnant'] = True
        
        pop.clear()
        pop = copy.deepcopy(newPop)
    
    globalEvaluationCounter += trialLoopCounter
    
print("--- %s seconds ---" % (time.time() - start_time))
print('{} solved out of {} trials'.format(solvedCounter, maxTestTrials))    
print('Total Number of evaluations overall {}'.format(globalEvaluationCounter))
print('Average evaluation count per trial: {}'.format(globalEvaluationCounter / maxTestTrials))
window.close()
env.close()
    
with open('../data/bestGenome_cartpole.dictionary', 'wb') as dictionary_file:
    pickle.dump(bestGenomeDictionary, dictionary_file)        

Beginning Trial #1
Beginning Generation #1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
Beginning Generation #2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
1

C:\Users\valen\anaconda3\lib\site-packages\ipykernel_launcher.py:284: RuntimeWarning: invalid value encountered in double_scalars


3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
mutation genome connections !
Beginning Generation #3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
Found an optimal genome. testing it...


In [ ]:
# load the networks that solved the problem !
with open('data/bestGenome_cartpole.dictionary', 'rb') as dictionary_file:
    bestGenomeDictionary = pickle.load(dictionary_file)

In [ ]:
# test bench 

env = gym.make(envName)

testBenchTotalFitness = 0

for i in range(100):
    t = 0
    bestGenome.resetFitness()
    observation = env.reset()
    while(True):
        t+=1
        env.render()
        action = bestGenome.evaluateAction(observation)
        observation, reward, done, info = env.step(action)
        bestGenome.fitness += reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            testBenchTotalFitness+=bestGenome.fitness
            print("Genome fitness: {}".format(bestGenome.fitness))
            break

print('average fitness across 100 trials:{}'.format(testBenchTotalFitness/100))
env.close()